In [114]:

from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField, StringType, IntegerType
from pyspark.sql.functions import max, desc

In [85]:
spark = SparkSession.builder.master("local[6]") \
                    .appName('intro_to_spark') \
                    .getOrCreate()

myRange = spark.range(1000).toDF("number")

In [9]:

# COMMAND ----------

divisBy2 = myRange.where("number % 2 = 0")


In [86]:


# COMMAND ----------
schema = StructType([
      StructField("DEST_COUNTRY_NAME", StringType(),True), 
      StructField("ORIGIN_COUNTRY_NAME", StringType(),True), 
      StructField("count", IntegerType(), True)
      ])
flightData2015 = spark\
  .read\
  .option("inferSchema", "true")\
  .option("header", "true")\
  .option("delimiter", ",")\
  .schema(schema)\
  .csv("../data/flight-data/csv/2015-summary.csv")



In [91]:
# COMMAND ----------

flightData2015.createOrReplaceTempView("flight_data_2015")

flightData2015.sort("count").take(2)

[Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Singapore', count=1),
 Row(DEST_COUNTRY_NAME='Moldova', ORIGIN_COUNTRY_NAME='United States', count=1)]

In [12]:

# COMMAND ----------

sqlWay = spark.sql("""
SELECT DEST_COUNTRY_NAME, count(1)
FROM flight_data_2015
GROUP BY DEST_COUNTRY_NAME
""")

dataFrameWay = flightData2015\
  .groupBy("DEST_COUNTRY_NAME")\
  .count()



In [13]:
sqlWay.explain()


== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- HashAggregate(keys=[DEST_COUNTRY_NAME#28], functions=[count(1)])
   +- Exchange hashpartitioning(DEST_COUNTRY_NAME#28, 200), ENSURE_REQUIREMENTS, [id=#33]
      +- HashAggregate(keys=[DEST_COUNTRY_NAME#28], functions=[partial_count(1)])
         +- FileScan csv [DEST_COUNTRY_NAME#28] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(1 paths)[file:/Users/Vivek_Goyal/dev/core-codecommit/repos/ds/Spark-The-Definit..., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string>




In [14]:
dataFrameWay.explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- HashAggregate(keys=[DEST_COUNTRY_NAME#28], functions=[count(1)])
   +- Exchange hashpartitioning(DEST_COUNTRY_NAME#28, 200), ENSURE_REQUIREMENTS, [id=#46]
      +- HashAggregate(keys=[DEST_COUNTRY_NAME#28], functions=[partial_count(1)])
         +- FileScan csv [DEST_COUNTRY_NAME#28] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(1 paths)[file:/Users/Vivek_Goyal/dev/core-codecommit/repos/ds/Spark-The-Definit..., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string>




In [18]:
spark.sql("SELECT max(count) FROM flight_data_2015").take(1)


[Row(max(count)=370002)]

In [78]:
flightData2015.printSchema()

root
 |-- DEST_COUNTRY_NAME: string (nullable = true)
 |-- ORIGIN_COUNTRY_NAME: string (nullable = true)
 |-- count: integer (nullable = true)



In [97]:
spark.sql("SELECT * FROM flight_data_2015 WHERE count in (SELECT max(count) FROM flight_data_2015)").explain()


== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- BroadcastHashJoin [count#382], [max(count)#429], LeftSemi, BuildRight, false
   :- FileScan csv [DEST_COUNTRY_NAME#380,ORIGIN_COUNTRY_NAME#381,count#382] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(1 paths)[file:/Users/Vivek_Goyal/dev/core-codecommit/repos/ds/Spark-The-Definit..., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string,ORIGIN_COUNTRY_NAME:string,count:int>
   +- BroadcastExchange HashedRelationBroadcastMode(List(cast(input[0, int, true] as bigint)),false), [id=#579]
      +- HashAggregate(keys=[], functions=[max(count#432)])
         +- Exchange SinglePartition, ENSURE_REQUIREMENTS, [id=#576]
            +- HashAggregate(keys=[], functions=[partial_max(count#432)])
               +- FileScan csv [count#432] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(1 paths)[file:/Users/Vivek_Goyal/dev/core-codecommit/repos/ds/Spark-The-De

In [113]:
spark.sql("""
            SELECT DEST_COUNTRY_NAME, sum(count) as total_count 
            FROM flight_data_2015 
            GROUP BY DEST_COUNTRY_NAME 
            ORDER BY sum(count) DESC 
            LIMIT 5
        """).show()

+-----------------+-----------+
|DEST_COUNTRY_NAME|total_count|
+-----------------+-----------+
|    United States|     411352|
|           Canada|       8399|
|           Mexico|       7140|
|   United Kingdom|       2025|
|            Japan|       1548|
+-----------------+-----------+



In [ ]:
flightData2015 \
    .groupBy("DEST_COUNTRY_NAME") \
    .sum("count") \
    .withColumnRenamed("sum(count)", "total_count") \
    .sort(desc("total_count")) \
    .limit(5) \
    .show()
